In [ ]:
import pandas as pd
import numpy as np
import re
import os
from pathlib import Path
from datetime import datetime
import unicodedata

# =========================
# CONFIGURAÇÕES
# =========================
PASTA_ORIGEM = Path(r'2 - CONVÊNIOS')
NOME_PLANILHA = 'Planilha de PJES'
COMPETENCIA_MANUAL = "2026-01"
erros = []  # Para registrar erros durante o processamento

# =========================
# FUNÇÃO PARA REGISTRAR ERROS
# =========================
def registrar_erro(msg_erro):
    erros.append(msg_erro)
    print(f"⚠️ {msg_erro}")

# =========================
# MAPA COMPLETO DE NORMALIZAÇÃO DE CARGOS
# =========================
MAPA_NORMALIZACAO_CARGOS = {
    # Abreviações comuns
    'SD': 'SOLDADO',
    'SDS': 'SOLDADOS',
    'SOLD': 'SOLDADO',
    'SOLDAD': 'SOLDADO',
    'CB': 'CABO',
    'CAB': 'CABO',
    'SGT': 'SARGENTO',
    'ST': 'SARGENTO',
    'SARG': 'SARGENTO',
    
    # Sargentos com números ordinais
    '3º SGT': 'TERCEIRO SARGENTO',
    '3 SGT': 'TERCEIRO SARGENTO',
    '3º SARGENTO': 'TERCEIRO SARGENTO',
    '3 SARGENTO': 'TERCEIRO SARGENTO',
    '3° SGT': 'TERCEIRO SARGENTO',
    '3° SARGENTO': 'TERCEIRO SARGENTO',
    
    '2º SGT': 'SEGUNDO SARGENTO',
    '2 SGT': 'SEGUNDO SARGENTO',
    '2º SARGENTO': 'SEGUNDO SARGENTO',
    '2 SARGENTO': 'SEGUNDO SARGENTO',
    '2° SGT': 'SEGUNDO SARGENTO',
    '2° SARGENTO': 'SEGUNDO SARGENTO',
    
    '1º SGT': 'PRIMEIRO SARGENTO',
    '1 SGT': 'PRIMEIRO SARGENTO',
    '1º SARGENTO': 'PRIMEIRO SARGENTO',
    '1 SARGENTO': 'PRIMEIRO SARGENTO',
    '1° SGT': 'PRIMEIRO SARGENTO',
    '1° SARGENTO': 'PRIMEIRO SARGENTO',
    
    # Subtenente
    'SUBTEN': 'SUBTENENTE',
    'SUB TEN': 'SUBTENENTE',
    'SUB-TEN': 'SUBTENENTE',
    'STEN': 'SUBTENENTE',
    
    # Aspirante
    'ASP': 'ASPIRANTE A OFICIAL',
    'ASP A OF': 'ASPIRANTE A OFICIAL',
    'ASPIR': 'ASPIRANTE A OFICIAL',
    'ASPIRANTE': 'ASPIRANTE A OFICIAL',
    
    # Tenentes
    '2º TEN': 'SEGUNDO TENENTE',
    '2 TEN': 'SEGUNDO TENENTE',
    '2º TENENTE': 'SEGUNDO TENENTE',
    '2 TENENTE': 'SEGUNDO TENENTE',
    '2° TEN': 'SEGUNDO TENENTE',
    '2° TENENTE': 'SEGUNDO TENENTE',
    
    '1º TEN': 'PRIMEIRO TENENTE',
    '1 TEN': 'PRIMEIRO TENENTE',
    '1º TENENTE': 'PRIMEIRO TENENTE',
    '1 TENENTE': 'PRIMEIRO TENENTE',
    '1° TEN': 'PRIMEIRO TENENTE',
    '1° TENENTE': 'PRIMEIRO TENENTE',
    
    # Capitão
    'CAP': 'CAPITÃO',
    'CAPIT': 'CAPITÃO',
    'CAPITAO': 'CAPITÃO',
    'CAPT': 'CAPITÃO',
    
    # Major
    'MAJ': 'MAJOR',
    
    # Tenente Coronel
    'TC': 'TENENTE CORONEL',
    'TEN CEL': 'TENENTE CORONEL',
    'TEN COR': 'TENENTE CORONEL',
    'TEN-COR': 'TENENTE CORONEL',
    
    # Coronel
    'CEL': 'CORONEL',
    'COR': 'CORONEL',
    
    # Polícia Civil / Perícia
    'DEL': 'DELEGADO',
    'DELEG': 'DELEGADO',
    'DELEGADO DE POLICIA': 'DELEGADO DE POLÍCIA CIVIL',
    'DELEGADO DE POL CIVIL': 'DELEGADO DE POLÍCIA CIVIL',
    'DELEGADO PC': 'DELEGADO DE POLÍCIA CIVIL',
    
    'PERITO': 'PERITO CRIMINAL',
    'PER CRIM': 'PERITO CRIMINAL',
    'PERITO CRIM': 'PERITO CRIMINAL',
    
    'PAPILOSCOPISTA': 'PERITO PAPILOSCOPISTA',
    'PER PAP': 'PERITO PAPILOSCOPISTA',
    'PERITO PAP': 'PERITO PAPILOSCOPISTA',
    
    'MED LEG': 'MÉDICO LEGISTA',
    'MEDICO LEG': 'MÉDICO LEGISTA',
    'MED LEGISTA': 'MÉDICO LEGISTA',
    
    'AGENTE POL': 'AGENTE DE POLÍCIA',
    'AGENTE DE POL': 'AGENTE DE POLÍCIA',
    'AG POLICIA': 'AGENTE DE POLÍCIA',
    
    'ESCRIV': 'ESCRIVÃO',
    'ESCRIVAO': 'ESCRIVÃO',
    'ESCRIVÃO DE POL': 'ESCRIVÃO DE POLÍCIA',
    
    'AG PERICIA': 'AGENTE DE PERÍCIA CRIMINAL',
    'AGENTE PER': 'AGENTE DE PERÍCIA CRIMINAL',
    
    'AG MED LEGAL': 'AGENTE DE MEDICINA LEGAL',
    'AGENTE MED LEGAL': 'AGENTE DE MEDICINA LEGAL',
    
    # Adicional: remover "DE" desnecessário
    'DE POLICIA': 'POLÍCIA',
    'DE POLÍCIA': 'POLÍCIA',
    'DE PERICIA': 'PERÍCIA',
    'DE PERÍCIA': 'PERÍCIA',
    'DE MEDICINA': 'MEDICINA',
}

# Mapa de valores por cota (após normalização)
MAPA_VALORES_COTA = {
    # Praças - R$ 200
    "SOLDADO": 200,
    "CABO": 200,
    "TERCEIRO SARGENTO": 200,
    "SEGUNDO SARGENTO": 200,
    "PRIMEIRO SARGENTO": 200,
    "SUBTENENTE": 200,
    "AGENTE DE POLÍCIA": 200,
    "AGENTE DE POLÍCIA CIVIL": 200,
    "ESCRIVÃO DE POLÍCIA": 200,
    "ESCRIVÃO": 200,
    "AGENTE DE PERÍCIA CRIMINAL": 200,
    "AGENTE DE MEDICINA LEGAL": 200,
    
    # Oficiais - R$ 300
    "ASPIRANTE A OFICIAL": 300,
    "SEGUNDO TENENTE": 300,
    "PRIMEIRO TENENTE": 300,
    "CAPITÃO": 300,
    "MAJOR": 300,
    "TENENTE CORONEL": 300,
    "CORONEL": 300,
    "DELEGADO DE POLÍCIA CIVIL": 300,
    "DELEGADO": 300,
    "PERITO CRIMINAL": 300,
    "PERITO PAPILOSCOPISTA": 300,
    "MÉDICO LEGISTA": 300,
}

# =========================
# FUNÇÕES DE NORMALIZAÇÃO DE CARGOS
# =========================
def normalizar_texto(texto):
    """Normaliza texto: remove acentos, caracteres especiais, padroniza"""
    if pd.isna(texto):
        return ''
    
    texto = str(texto).upper().strip()
    
    # Remove acentos
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitui símbolos de grau
    texto = texto.replace('º', ' ').replace('°', ' ').replace('ª', ' ')
    
    # Remove caracteres especiais, mantendo letras, números e espaços
    texto = re.sub(r'[^A-Z0-9\s]', ' ', texto)
    
    # Remove múltiplos espaços
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    return texto

def normalizar_cargo_completo(cargo):
    """Normaliza cargo substituindo todas as abreviações"""
    if pd.isna(cargo):
        return 'NÃO INFORMADO'
    
    # Primeiro, normalizar o texto
    cargo_norm = normalizar_texto(cargo)
    
    # print(f"  Debug - Cargo original: '{cargo}' -> Normalizado: '{cargo_norm}'")
    
    # Substituir abreviações (em ordem de especificidade)
    # Primeiro procurar por padrões específicos
    for abreviacao, completo in MAPA_NORMALIZACAO_CARGOS.items():
        abreviacao_norm = normalizar_texto(abreviacao)
        completo_norm = normalizar_texto(completo)
        
        # Verificar se a abreviação está no cargo (como palavra completa)
        if abreviacao_norm and cargo_norm == abreviacao_norm:
            # print(f"  Debug - Substituição exata: '{abreviacao_norm}' -> '{completo_norm}'")
            cargo_norm = completo_norm
            break
        elif abreviacao_norm in cargo_norm.split():
            # Substituir a abreviação dentro do texto
            cargo_norm = re.sub(rf'\b{re.escape(abreviacao_norm)}\b', completo_norm, cargo_norm)
            # print(f"  Debug - Substituição parcial: '{abreviacao_norm}' -> '{completo_norm}' no '{cargo_norm}'")
    
    # Garantir que termos importantes estejam completos
    termo_substituicoes = {
        'POLICIA': 'POLÍCIA',
        'PERICIA': 'PERÍCIA',
        'MEDICO': 'MÉDICO',
        'ESCRIVAO': 'ESCRIVÃO',
        'CAPITAO': 'CAPITÃO',
    }
    
    for termo_errado, termo_correto in termo_substituicoes.items():
        termo_errado_norm = normalizar_texto(termo_errado)
        termo_correto_norm = normalizar_texto(termo_correto)
        cargo_norm = re.sub(rf'\b{re.escape(termo_errado_norm)}\b', termo_correto_norm, cargo_norm)
    
    # Formatar para título (primeira letra maiúscula em cada palavra)
    palavras = cargo_norm.split()
    cargo_formatado = ' '.join([p.capitalize() for p in palavras])
    
    # Correções específicas para termos compostos
    correcoes_especificas = {
        'De': 'de',
        'Da': 'da',
        'Do': 'do',
        'A': 'a',
        'E': 'e',
        'O': 'o',
    }
    
    for palavra, correcao in correcoes_especificas.items():
        cargo_formatado = re.sub(rf'\b{palavra}\b', correcao, cargo_formatado)
    
    # Capitalizar primeira letra da string
    if cargo_formatado:
        cargo_formatado = cargo_formatado[0].upper() + cargo_formatado[1:]
    
    # print(f"  Debug - Cargo final: '{cargo_formatado}'")
    return cargo_formatado

def obter_valor_por_cota_normalizado(cargo_normalizado):
    """Retorna o valor da cota baseado no cargo normalizado"""
    if not cargo_normalizado or cargo_normalizado == 'NÃO INFORMADO':
        return 200  # Valor padrão
    
    cargo_upper = cargo_normalizado.upper()
    
    # Procurar correspondência exata
    for cargo_padrao, valor in MAPA_VALORES_COTA.items():
        if cargo_padrao.upper() in cargo_upper or cargo_upper in cargo_padrao.upper():
            # print(f"  Debug - Valor encontrado para '{cargo_normalizado}': R$ {valor}")
            return valor
    
    # Heurística baseada em palavras-chave
    palavras_oficial = ['ASPIRANTE', 'TENENTE', 'CAPITÃO', 'MAJOR', 'CORONEL', 
                       'DELEGADO', 'PERITO', 'MÉDICO', 'PAPILOSCOPISTA']
    
    palavras_praca = ['SOLDADO', 'CABO', 'SARGENTO', 'SUBTENENTE', 
                     'AGENTE', 'ESCRIVÃO']
    
    cargo_upper_sem_acentos = unicodedata.normalize('NFKD', cargo_upper).encode('ASCII', 'ignore').decode('ASCII')
    
    for palavra in palavras_oficial:
        if palavra in cargo_upper_sem_acentos:
            # print(f"  Debug - Oficial detectado em '{cargo_normalizado}': R$ 300")
            return 300
    
    for palavra in palavras_praca:
        if palavra in cargo_upper_sem_acentos:
            # print(f"  Debug - Praça detectado em '{cargo_normalizado}': R$ 200")
            return 200
    
    # print(f"  Debug - Cargo não encontrado: '{cargo_normalizado}', usando valor padrão: R$ 200")
    return 200  # Valor padrão

# =========================
# NORMALIZAÇÃO DE COLUNAS
# =========================
def padronizar_colunas(df):
    def normalizar(col):
        col = col.strip().upper()
        col = unicodedata.normalize('NFKD', col).encode('ASCII', 'ignore').decode('ASCII')
        col = re.sub(r'[^A-Z0-9 ]+', '', col)
        col = re.sub(r'\s+', ' ', col)
        return col

    df.columns = [normalizar(c) for c in df.columns]

    mapa = {
        'NOME': ['NOME EXTENSO', 'NOME COMPLETO'],
        'CARGO': ['CARGO', 'FUNCAO', 'POSTO GRADUACAO'],
        'OPERATIVA': ['OPERATIVA', 'UNIDADE', 'LOTACAO'],
        'DT TERMINO': ['DT TERMINO', 'DATA TERMINO', 'FIM'],
        'DT INICIO': ['DT INICIO', 'DATA INICIO'],
        'QTDCOTA': ['QTDCOTA', 'QTD COTA', 'COTA'],
        'TITULO': ['TITULO', 'LOCAL', 'EVENTO'],
        'MATRICULA': ['MATRICULA'],
        'VALOR': ['VALOR', 'VALOR TOTAL', 'VALOR A PAGAR', 'VLR']
    }

    for padrao, sinonimos in mapa.items():
        for col in sinonimos:
            if col in df.columns:
                df.rename(columns={col: padrao}, inplace=True)
                break

    return df

# =========================
# COMPETÊNCIA / EXERCÍCIO
# =========================
MAPA_MESES = {
    1: 'JANEIRO',
    2: 'FEVEREIRO',
    3: 'MARÇO',
    4: 'ABRIL',
    5: 'MAIO',
    6: 'JUNHO',
    7: 'JULHO',
    8: 'AGOSTO',
    9: 'SETEMBRO',
    10: 'OUTUBRO',
    11: 'NOVEMBRO',
    12: 'DEZEMBRO'
}

def extrair_competencia(nome_arquivo):
    # Se houver competência manual definida, usa ela
    if COMPETENCIA_MANUAL:
        try:
            ano, mes = COMPETENCIA_MANUAL.split("-")
            return datetime(int(ano), int(mes), 1)
        except Exception:
            print("⚠️ COMPETENCIA_MANUAL inválida. Use o formato AAAA-MM, ex: 2026-01")
            return datetime.today().replace(day=1)

    # Caso contrário, continua usando o nome do arquivo (modo automático)
    nome = nome_arquivo.upper()
    mes = None
    ano = None

    for m in MAPA_MESES:
        if m in nome:
            mes = MAPA_MESES[m]
            break

    ano_match = re.search(r'20\d{2}', nome)
    if ano_match:
        ano = int(ano_match.group())

    if mes and ano:
        return datetime(ano, mes, 1)

    return datetime.today().replace(day=1)

# =========================
# FUNÇÃO PARA CALCULAR VERBA
# =========================

def calcular_verba(data_fim, competencia):
    if pd.isna(data_fim):   
        return ''
    data_fim = pd.to_datetime(data_fim) 
    competencia = pd.to_datetime(competencia) 
    inicio_comp = competencia.replace(day=1)
    if data_fim.year == competencia.year and data_fim.month == competencia.month:
        return 223
    elif data_fim < inicio_comp:
        return 423
    return ''

# =========================
# FUNÇÃO PARA CALCULAR VALOR
# =========================
def calcular_valor_com_cota_normalizado(row):
    """Calcula o valor baseado na cota e no cargo normalizado"""
    try:
        # Primeiro, tentar usar o valor existente na planilha
        if 'VALOR' in row.index and not pd.isna(row['VALOR']):
            try:
                valor = float(row['VALOR'])
                if valor > 0:
                    return valor
            except:
                pass
        
        # Se não houver valor ou for inválido, calcular baseado na cota e cargo normalizado
        if 'QTDCOTA' in row.index and 'CARGO NORMALIZADO' in row.index:
            cota = pd.to_numeric(row['QTDCOTA'], errors='coerce')
            cargo_normalizado = row['CARGO NORMALIZADO']
            
            if not pd.isna(cota) and cota > 0:
                valor_por_cota = obter_valor_por_cota_normalizado(cargo_normalizado)
                valor_total = cota * valor_por_cota
                return round(valor_total, 2)
        
        return 0.0
    except Exception as e:
        print(f"  Erro ao calcular valor: {e}")
        return 0.0

# =========================
# PROCESSAMENTO DE ARQUIVO COM NORMALIZAÇÃO
# =========================
def processar_arquivo(arquivo):
    try:
        print(f"📄 Processando: {arquivo.name}")
        
        # Extrair competência do nome do arquivo
        competencia_data = extrair_competencia(arquivo.name)
        
        # Ler o arquivo Excel
        df = pd.read_excel(
            arquivo,
            sheet_name=NOME_PLANILHA,
            skiprows=9,
            usecols="A:L",
            engine='openpyxl'
        )
        
        # Remover linhas completamente vazias
        df.dropna(how='all', inplace=True)
        
        if len(df) == 0:
            print(f"   ⚠️ Arquivo vazio ou sem dados após linha 9")
            return None
        
        # Padronizar colunas
        df = padronizar_colunas(df)
        
        # Tratamento de dados
        if 'DT TERMINO' in df.columns:
            df['DT TERMINO'] = pd.to_datetime(df['DT TERMINO'], errors='coerce', dayfirst=True)
        else:
            df['DT TERMINO'] = pd.NaT
        
        # NORMALIZAR CARGOS
        print(f"   🔧 Normalizando cargos...")
        if 'CARGO' in df.columns:
            # Manter o cargo original para referência
            df['CARGO ORIGINAL'] = df['CARGO'].astype(str).str.strip()
            
            # Aplicar normalização completa
            df['CARGO NORMALIZADO'] = df['CARGO'].apply(normalizar_cargo_completo)
            
            # Usar o cargo normalizado como principal
            df['CARGO'] = df['CARGO NORMALIZADO']
            
            # Estatísticas de normalização
            cargos_unicos = df['CARGO ORIGINAL'].nunique()
            cargos_normais_unicos = df['CARGO NORMALIZADO'].nunique()
            print(f"   📊 Cargos únicos: {cargos_unicos} → {cargos_normais_unicos} (normalizados)")
        else:
            df['CARGO'] = 'NÃO INFORMADO'
            df['CARGO ORIGINAL'] = 'NÃO INFORMADO'
            df['CARGO NORMALIZADO'] = 'NÃO INFORMADO'
        
        if 'QTDCOTA' in df.columns:
            df['QTDCOTA'] = pd.to_numeric(df['QTDCOTA'], errors='coerce').fillna(0)
        else:
            df['QTDCOTA'] = 0
        
        # Calcular VALOR usando cargos normalizados
        print(f"   💰 Calculando valores...")
        df['VALOR'] = df.apply(calcular_valor_com_cota_normalizado, axis=1)
        
        df['EXERCÍCIO'] = competencia_data.year 
        
        mes_numero = competencia_data.month
        df['COMPETÊNCIA'] = MAPA_MESES.get(mes_numero, '').upper()

        df['ORIGEM COTA'] = 'DECRETO'
        df['ORIGEM DA INFORMAÇÃO'] = arquivo.name.replace('.xlsx', '')
        
        # Verificar se as colunas existem antes de usá-las
        if 'OPERATIVA' in df.columns:
            df['OPERATIVA QUE PRESTOU SERVIÇO'] = df['OPERATIVA']
        else:
            df['OPERATIVA QUE PRESTOU SERVIÇO'] = ''
            
        if 'TITULO' in df.columns:
            df['LOCAL DA PRESTAÇÃO DO SERVIÇO'] = df['TITULO']
        else:
            df['LOCAL DA PRESTAÇÃO DO SERVIÇO'] = ''
        
        df['COTA'] = df['QTDCOTA'].astype(int)
        
        if 'DT TERMINO' in df.columns:
            df['VERBA'] = df['DT TERMINO'].apply(lambda x: calcular_verba(x, competencia_data))
        else:
            df['VERBA'] = ''
        
        df['SITUAÇÃO'] = 'ATIVO'
        
        # Tratar matrícula
        if 'MATRICULA' in df.columns:
            df['MATRICULA'] = df['MATRICULA'].astype(str).str.replace(r'\.0$', '', regex=True)
        else:
            df['MATRICULA'] = ''
        
        # Verificar qual coluna de nome usar
        if 'NOME' in df.columns:
            df['NOME'] = df['NOME']
        elif 'NOME EXTENSO' in df.columns:
            df['NOME'] = df['NOME EXTENSO']
        elif 'NOME COMPLETO' in df.columns:
            df['NOME'] = df['NOME COMPLETO']
        else:
            # Tentar encontrar coluna que contenha 'NOME'
            coluna_nome = [c for c in df.columns if 'NOME' in c]
            if coluna_nome:
                df['NOME'] = df[coluna_nome[0]]
            else:
                df['NOME'] = ''
        
        # Adicionar valor por cota para análise
        df['VALOR POR COTA'] = df['CARGO NORMALIZADO'].apply(obter_valor_por_cota_normalizado)
        
        # Estatísticas do processamento
        valor_total = df['VALOR'].sum()
        print(f"   ✅ {len(df)} registros processados")
        print(f"   💰 Valor total calculado: R$ {valor_total:,.2f}")
        print(f"   📊 Valor médio por cota: R$ {df['VALOR POR COTA'].mean():.2f}")
        
        # Mostrar alguns exemplos de normalização
        if 'CARGO ORIGINAL' in df.columns and 'CARGO NORMALIZADO' in df.columns:
            exemplos = df[['CARGO ORIGINAL', 'CARGO NORMALIZADO', 'VALOR POR COTA']].drop_duplicates().head(3)
            print(f"   📝 Exemplos de normalização:")
            for idx, row in exemplos.iterrows():
                print(f"     '{row['CARGO ORIGINAL']}' → '{row['CARGO NORMALIZADO']}' (R$ {row['VALOR POR COTA']}/cota)")
        
        # Organizar colunas finais
        colunas_finais = [
            'EXERCÍCIO', 'COMPETÊNCIA', 'ORIGEM COTA', 'ORIGEM DA INFORMAÇÃO',
            'OPERATIVA QUE PRESTOU SERVIÇO', 'LOCAL DA PRESTAÇÃO DO SERVIÇO',
            'MATRICULA', 'NOME', 'COTA', 'VERBA', 'CARGO', 'SITUAÇÃO', 'VALOR',
            'VALOR POR COTA', 'CARGO NORMALIZADO', 'CARGO ORIGINAL'
        ]
        
        # Garantir que todas as colunas existam
        for col in colunas_finais:
            if col not in df.columns:
                df[col] = ''
                
        df_final = df[colunas_finais].copy()

        # juntar colunas originais com as finais
        df_completo = pd.concat([df, df_final], axis=1)
        
        # remover duplicadas mantendo a primeira ocorrência
        df_completo = df_completo.loc[:, ~df_completo.columns.duplicated()]
        
        return df_final, df_completo

                
    except Exception as e:
        msg_erro = f"Erro ao processar o arquivo '{arquivo.name}': {str(e)}"
        registrar_erro(msg_erro)
        return None

# =========================
# RELATÓRIO DE NORMALIZAÇÃO
# =========================
def gerar_relatorio_normalizacao(df):
    """Gera um relatório detalhado da normalização de cargos"""
    if df is None or len(df) == 0:
        return
    
    print(f"\n📋 RELATÓRIO DE NORMALIZAÇÃO DE CARGOS")
    print("=" * 70)
    
    # Agrupar cargos originais e normalizados
    relatorio_cargos = df.groupby(['CARGO ORIGINAL', 'CARGO NORMALIZADO', 'VALOR POR COTA']).agg({
        'NOME': 'count',
        'VALOR': 'sum',
        'COTA': 'sum'
    }).reset_index()
    
    relatorio_cargos = relatorio_cargos.rename(columns={
        'NOME': 'QUANTIDADE',
        'VALOR': 'TOTAL VALOR (R$)',
        'COTA': 'TOTAL COTAS'
    })
    
    # Ordenar por quantidade
    relatorio_cargos = relatorio_cargos.sort_values('QUANTIDADE', ascending=False)
    
    print("Principais transformações:")
    print("-" * 70)
    for idx, row in relatorio_cargos.head(10).iterrows():
        print(f"'{row['CARGO ORIGINAL']}' → '{row['CARGO NORMALIZADO']}'")
        print(f"  R$ {row['VALOR POR COTA']}/cota | {row['QUANTIDADE']} pessoas | {row['TOTAL COTAS']} cotas | R$ {row['TOTAL VALOR (R$)']:,.2f}")
        print()
    
    # Resumo por valor por cota
    print("\n📊 RESUMO POR VALOR DE COTA:")
    print("-" * 50)
    resumo_valor = df.groupby('VALOR POR COTA').agg({
        'NOME': 'count',
        'VALOR': 'sum',
        'COTA': 'sum'
    }).reset_index()
    
    for idx, row in resumo_valor.iterrows():
        valor_cota = row['VALOR POR COTA']
        qtd = row['NOME']
        total_valor = row['VALOR']
        total_cotas = row['COTA']
        print(f"R$ {valor_cota}/cota: {qtd} pessoas, {total_cotas} cotas, R$ {total_valor:,.2f}")

# =========================
# PROCESSAMENTO PRINCIPAL
# =========================
def processar_pasta(pasta):
    completos = []
    dfs = []
    
    # Verificar se a pasta existe
    if not pasta.exists():
        print(f"❌ Pasta não encontrada: {pasta}")
        return None
    
    # Processar todos os arquivos Excel na pasta
    arquivos = list(pasta.glob('*.xlsx'))
    print(f"📁 Encontrados {len(arquivos)} arquivos Excel em {pasta}")
    print("=" * 60)
    
    for arquivo in arquivos:
        resultado = processar_arquivo(arquivo)
    
        if resultado is not None:
            resultado_df, completo_df = resultado
            dfs.append(resultado_df)
            completos.append(completo_df)
    
        print("-" * 60)

    
    # Combinar todos os DataFrames
    if dfs:
        data = pd.concat(dfs, ignore_index=True)
        dados_completos = pd.concat(completos, ignore_index=True)
        data['COTA'] = pd.to_numeric(data['COTA'], errors='coerce').fillna(0)
        data['VALOR'] = pd.to_numeric(data['VALOR'], errors='coerce').fillna(0)
        
        # Gerar relatório de normalização
        gerar_relatorio_normalizacao(data)
        
        # Agrupar dados para o resultado final
        colunas_agrupamento = [
            'EXERCÍCIO', 'COMPETÊNCIA', 'ORIGEM COTA', 'ORIGEM DA INFORMAÇÃO',
            'OPERATIVA QUE PRESTOU SERVIÇO', 'LOCAL DA PRESTAÇÃO DO SERVIÇO',
            'MATRICULA', 'NOME', 'VERBA', 'CARGO NORMALIZADO', 'SITUAÇÃO'
        ]
        
        # Garantir que todas as colunas de agrupamento existam
        colunas_agrupamento = [col for col in colunas_agrupamento if col in data.columns]
        
        data_agrupado = data.groupby(colunas_agrupamento, as_index=False).agg({
            'COTA': 'sum',
            'VALOR': 'sum',
            'VALOR POR COTA': 'first',
            'CARGO ORIGINAL': lambda x: '; '.join(set(x)) if len(set(x)) <= 3 else f"{len(set(x))} variações"
        }).sort_values(by='VALOR', ascending=False)
        
        return data_agrupado, dados_completos
    else:
        print("⚠️ Nenhum arquivo foi processado com sucesso.")
        return None, None

# =========================
# EXECUÇÃO PRINCIPAL
# =========================
if __name__ == "__main__":
    print("🚀 Iniciando processamento de arquivos PJES com normalização de cargos...")
    print("=" * 70)
    
    resultado, dados_completos = processar_pasta(PASTA_ORIGEM)
    
    if resultado is not None:
        print(f"\n" + "=" * 70)
        print("✅ PROCESSAMENTO CONCLUÍDO COM NORMALIZAÇÃO DE CARGOS!")
        print("=" * 70)
        print(f"📊 Total de registros únicos: {len(resultado):,}")
        print(f"📊 Total de cotas: {resultado['COTA'].sum():,.0f}")
        print(f"💰 Total de valor: R$ {resultado['VALOR'].sum():,.2f}")
        
        # Estatísticas por tipo de cargo
        print(f"\n📈 DISTRIBUIÇÃO POR VALOR DE COTA:")
        for valor_cota in sorted(resultado['VALOR POR COTA'].unique()):
            subset = resultado[resultado['VALOR POR COTA'] == valor_cota]
            print(f"   • R$ {valor_cota}/cota: {len(subset):,} pessoas, R$ {subset['VALOR'].sum():,.2f}")
        
        # Salvar resultado consolidado
        arquivo_saida = PASTA_ORIGEM / 'PJES_CONSOLIDADO_NORMALIZADO.xlsx'
        resultado.to_excel(arquivo_saida, index=False)
        print(f"\n💾 Resultado consolidado salvo em: {arquivo_saida}")
        
        # Salvar dados completos
        if dados_completos is not None:
            arquivo_completo = PASTA_ORIGEM / 'PJES_DADOS_COMPLETOS_NORMALIZADOS.xlsx'
            dados_completos.to_excel(arquivo_completo, index=False)
            print(f"💾 Dados completos salvos em: {arquivo_completo}")
        
        # Mostrar amostra dos dados
        print(f"\n📋 AMOSTRA DOS DADOS NORMALIZADOS (5 maiores valores):")
        amostra = resultado[['NOME', 'CARGO NORMALIZADO', 'VALOR POR COTA', 'COTA', 'VALOR']].head()
        print(amostra.to_string(index=False))
        
        # Mostrar erros se houver
        if erros:
            print(f"\n⚠️ {len(erros)} erro(s) encontrado(s):")
            for i, erro in enumerate(erros[:3], 1):
                print(f"   {i}. {erro}")
            if len(erros) > 3:
                print(f"   ... e mais {len(erros) - 3} erros")
                
        # Salvar log de erros
        if erros:
            log_erros = PASTA_ORIGEM / 'PJES_ERROS_LOG.txt'
            with open(log_erros, 'w', encoding='utf-8') as f:
                f.write("LOG DE ERROS - PROCESSAMENTO PJES COM NORMALIZAÇÃO\n")
                f.write("=" * 60 + "\n")
                for erro in erros:
                    f.write(f"• {erro}\n")
            print(f"\n📝 Log de erros salvo em: {log_erros}")
    else:
        print("\n❌ Nenhum dado foi processado.")